In [1]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time
import re
import hashlib

import sqlite3
import csv
import os



In [2]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    #working_df=working_df.drop_duplicates()
    print(f'working on {hashtag}')
    index=len(working_df)
    for tweet in tweets: 
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        #print(working_df.loc[index,'created_at'])
        #working_df=working_df.drop_duplicates()
        index+=1
        working_df.to_csv(f'source/{hashtag}.csv', index=False)
        #time.sleep(1)
    working_df=working_df.drop_duplicates()
    working_df.to_csv(f'source/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
          
    return working_df

In [5]:
def get_tweets_from_general_tags(working_df, hashtag):
    #load and merge the general_tags
    print(f'working on {hashtag}')
    general_tags = ['#SaveLockwoodandCo', 'Lockwood and Co', 'Lockwood']
    general_tags_df=pd.DataFrame()
    for tag in general_tags:
        try:
            general_tags_df=pd.concat([general_tags_df, pd.read_csv(f'source/{tag}.csv')], ignore_index=True)
        except:
            pass
    general_tags_df =general_tags_df.drop_duplicates()
    print(f'total generaltags: {len(general_tags_df)}')
    #general_tags_df['full_text'] = general_tags_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    #Get the rows in the working_df that have the hashtag inside the full_text column
    hashtag_df = general_tags_df[general_tags_df['full_text'].apply(lambda x: hashtag.lower() in x.lower())]
    #hashtag_df = hashtag_df.reset_index(drop=True)
    hashtag_df['hashtags'] = hashtag
    working_df=pd.concat([working_df, hashtag_df], ignore_index=True)
    working_df=working_df.drop_duplicates()
    working_df['full_text'] = working_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    working_df.to_csv(f'source/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
  
    return working_df

In [6]:
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [65]:
hashtagsweek1 = ['#FridayNightatPortlandRow', '#HauntedWatchParty', '#WatchPartyatPortlandRow', '#HauntedbyaType3','#TogetherForLockwoodandCo','#PrimeForLockwoodandCo','#BringBackLockwoodandCo']
hashtagsweek2 = ['#GhostHuntersWatchParty', '#DisneyForLockwoodandCo', '#BBCforLockwoodandCo', '#AppleTVforLockwoodandCo', '#PrimeForLockwoodandCo', '#JustRecklessEnough']
hashtagsweek3 = ['#LockwoodGhostAuditions', '#ParamountForLockwoodandCo', '#ScullandCo','#RapiersReady', '#CaringforCarlyle', '#DEPRACisOnTheWay', '#BunsForBunchurch']
hashtagsweek4 = ['#CompleteFictionAppreciation', '#DisneySaveLockwood', '#ArtistryofLockwoodandCo', '#ParamountSaveLockwood', '#GhostStrike', '#LockwoodParallelFandoms', '#JustRecklessEnough']
hashtagsweek5 = ['#StroudsAppreciation', '#VoteLockwoodforNFA', '#PrimeSaveLockwood', '#ScreamingStaircase', '#DEPRACrollcall', '#LivingforLockwood', '#RapiersReady']
hashtagsweek6 = ['#LockNationAppreciation', '#LockNationArtistsandGiftsDay', '#LockNationEditorsDay', '#LockNationFicWritersDay', '#LockNationComediansDay', '#GhostLockAwards']
hashtagsweek7 = ['VoteLocwoodforNFA', '#LockwoodCastAppreciation', '#YouMeAndHerons', '#FishAndKipps', '#LockwoodDnDCampaign', '#TheIronTrio', '#VirtualThinkingCloth']
hashtagsall = hashtagsweek1+hashtagsweek2+hashtagsweek3+hashtagsweek4 + hashtagsweek5 + hashtagsweek6 + hashtagsweek7
hashtagsall = list(set(hashtagsall))

In [8]:

phrase = 'Lockwood and Co'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=phrase, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)

hashtag = '#SaveLockwoodandCo'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)


36254
working on Lockwood and Co


/tmp/ipykernel_786/3429170079.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)


created_at
2023-06-02 00:00:00+00:00      64
2023-06-03 00:00:00+00:00     313
2023-06-04 00:00:00+00:00     275
2023-06-05 00:00:00+00:00     443
2023-06-06 00:00:00+00:00     725
2023-06-07 00:00:00+00:00     922
2023-06-08 00:00:00+00:00    1159
2023-06-09 00:00:00+00:00    1710
2023-06-10 00:00:00+00:00    1937
2023-06-11 00:00:00+00:00    2756
2023-06-12 00:00:00+00:00     407
2023-06-13 00:00:00+00:00       0
2023-06-14 00:00:00+00:00       0
2023-06-15 00:00:00+00:00    3956
2023-06-16 00:00:00+00:00    5286
2023-06-17 00:00:00+00:00    3615
2023-06-18 00:00:00+00:00    3940
2023-06-19 00:00:00+00:00       0
2023-06-20 00:00:00+00:00       0
2023-06-21 00:00:00+00:00       0
2023-06-22 00:00:00+00:00       0
2023-06-23 00:00:00+00:00       0
2023-06-24 00:00:00+00:00     379
2023-06-25 00:00:00+00:00    1614
2023-06-26 00:00:00+00:00      14
Freq: D, Name: hashtags, dtype: int64
29515
29515
working on Lockwood and Co
created_at
2023-06-02 00:00:00+00:00      64
2023-06-03 00:00:

In [91]:
phrase = 'Lockwood'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=phrase, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)

working on Lockwood
created_at
2023-06-26 00:00:00+00:00    149
Freq: D, Name: hashtags, dtype: int64
149


In [93]:
hashtags = ['#LockNationArtistsandGiftsDay']
for hashtag in hashtags:
    try:
        working_df=pd.read_csv(f'source/{hashtag}.csv')
        #working_df = pd.read_csv('source/#SkullandCo.csv')
        print(len(working_df))
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
    tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
    working_df = get_tweets_dataframe(working_df, tweets, hashtag)
    working_df=working_df.drop_duplicates()
    print(len(working_df))
    time.sleep(60)

1
working on #LockNationArtistsandGiftsDay
created_at
2023-06-20 00:00:00+00:00    2
Freq: D, Name: hashtags, dtype: int64
1


In [92]:
hashtags = ['#SkullandCo']
for hashtag in hashtagsweek6:
    try:
        
        working_df=pd.read_csv(f'source/{hashtag}.csv')
        print('try')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
        print('except')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
        

try
working on #LockNationAppreciation
total generaltags: 81883
created_at
2023-06-18 00:00:00+00:00       4
2023-06-19 00:00:00+00:00    3670
2023-06-20 00:00:00+00:00    1174
2023-06-21 00:00:00+00:00      66
2023-06-22 00:00:00+00:00       8
2023-06-23 00:00:00+00:00       4
2023-06-24 00:00:00+00:00       0
2023-06-25 00:00:00+00:00      20
Freq: D, Name: hashtags, dtype: int64
try
working on #LockNationArtistsandGiftsDay


/tmp/ipykernel_786/2862647720.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashtag_df['hashtags'] = hashtag


total generaltags: 81883
created_at
2023-06-20 00:00:00+00:00    1
Freq: D, Name: hashtags, dtype: int64
try
working on #LockNationEditorsDay
total generaltags: 81883
created_at
2023-06-20 00:00:00+00:00      45
2023-06-21 00:00:00+00:00    5148
2023-06-22 00:00:00+00:00    1187
2023-06-23 00:00:00+00:00      51
2023-06-24 00:00:00+00:00       4
2023-06-25 00:00:00+00:00       4
Freq: D, Name: hashtags, dtype: int64
try
working on #LockNationFicWritersDay
total generaltags: 81883
created_at
2023-06-23 00:00:00+00:00    3299
2023-06-24 00:00:00+00:00     479
2023-06-25 00:00:00+00:00      14
Freq: D, Name: hashtags, dtype: int64
try
working on #LockNationComediansDay
total generaltags: 81883
created_at
2023-06-24 00:00:00+00:00    2894
2023-06-25 00:00:00+00:00     804
2023-06-26 00:00:00+00:00      12
Freq: D, Name: hashtags, dtype: int64
try
working on #GhostLockAwards
total generaltags: 81883
created_at
2023-06-25 00:00:00+00:00    4107
2023-06-26 00:00:00+00:00     495
Freq: D, Name

In [67]:
working_df=pd.DataFrame()
for hashtag in hashtagsall:
    try:
        working_df=pd.concat([working_df, pd.read_csv(f'source/{hashtag}.csv')], ignore_index=True)
    except:
        pass
len(working_df)

227730

In [68]:
working_df['full_text'] = working_df['full_text'].str.replace(r'"', '').replace(r'\n', ' ').replace(':', ' ').replace(',', '').replace('!', '')

In [69]:
# Create a new column 'retweet' with default value False
working_df['retweet'] = False

# Check if 'full_text' starts with 'RT ' and set 'retweet' column accordingly
working_df.loc[working_df['full_text'].str.startswith('RT '), 'retweet'] = True

#Remove the 'RT ' in the full_text column
working_df['full_text'] = working_df['full_text'].str.replace('RT ', '')

working_df['retweet'].value_counts()

False    128901
True      98829
Name: retweet, dtype: int64

In [70]:
def get_week_label(week_number):
    if week_number == 20:
        return 'week1'
    elif week_number == 21:
        return 'week2'
    elif week_number == 22:
        return 'week3'
    elif week_number == 23:
        return 'week4'
    elif week_number == 24:
        return 'week5'
    elif week_number == 25:
        return 'week6'
    else:
        return 'week1'


In [71]:
working_df['created_date'] = pd.to_datetime(working_df['created_at'], utc=True)
working_df['week'] = working_df['created_date'].dt.isocalendar().week
working_df['week'] = working_df['week'].apply(get_week_label)
working_df['week'].value_counts()

week4    47107
week1    45695
week3    45254
week5    37164
week2    36116
week6    16394
Name: week, dtype: int64

In [72]:
#Create an anonymized tweet_id
#working_df['unique_id'] = str(working_df['tweet_id']) + working_df['created_at'] + working_df['user'] + str(working_df['retweet'])
working_df['unique_id'] = working_df.apply(lambda row: str(row['tweet_id']) + row['created_at'] + row['user'] + str(row['retweet']), axis=1)
working_df['tweet_id'] = working_df['unique_id'].apply(anonymize)

In [73]:
hashtags_df = working_df[['tweet_id','hashtags']]
hashtags_df.to_csv('output/hashtags.csv', index=False)
working_df.value_counts('hashtags')

hashtags
#JustRecklessEnough              18885
#PrimeForLockwoodandCo           15252
#RapiersReady                    15224
#ParamountForLockwoodandCo        9958
#TogetherForLockwoodandCo         9416
#BunsForBunchurch                 9283
#ParamountSaveLockwood            9196
#CompleteFictionAppreciation      8824
#BringBackLockwoodandCo           7874
#LockwoodParallelFandoms          7759
#ScreamingStaircase               7198
#LivingforLockwood                6494
#LockwoodGhostAuditions           6465
#GhostStrike                      6350
#CaringforCarlyle                 6289
#DisneyForLockwoodandCo           5866
#HauntedbyaType3                  5811
#DEPRACisOnTheWay                 5425
#BBCforLockwoodandCo              5380
#WatchPartyatPortlandRow          5255
#FridayNightatPortlandRow         5216
#DEPRACrollcall                   5170
#DisneySaveLockwood               5022
#HauntedWatchParty                4838
#AppleTVforLockwoodandCo          4748
#LockNationAppre

In [74]:
# Get the tagged users from the full_text
working_df=working_df.drop_duplicates('tweet_id')
filter_out = ['@AppleSupport ','@AppleTV ','@AppleTVI ','@AppleTVPlus ','@AppleTVUK ','@BBC ', '@BBCOne ', '@BBCPlayer ','@BBCSounds ', '@BBCR1 ',
            '@BBCSounds ','@BBCiPlayer ','@BBCone ','@D ','@Di ','@Dis ','@Disn ', '@Disne ', '@Dinsey ','@DisneyChannel ','@DisneyHyperion ','@DisneyIT ', '@DisneyPlus ', '@DisneyChannel ',
            '@DisneyHyperion ','@DisneyPlusUK', '@DisneyPlus', '@HBO_UK ','@HBO ', '@hulu ','@NETFLIX ', '@netflix ', '@NETFLIXUK ', '@Netflix ', '@NetflixUK ','@ParamountPlus ', '@ParamaountUK ', '@ParamountPlusUK '
            '@Prime ', '@PrimeUK ', '@PrimeVideo ', '@PrimeVideo','@primevideouk ']
for filter in filter_out:
    working_df['full_text'] = working_df['full_text'].str.replace(filter, '')
working_df['tagged_users'] = working_df['full_text'].apply(lambda x: re.findall(r'@(\w+)', x))
# Remove the tagged users from the full_text
working_df['full_text'] = working_df['full_text'].str.replace(r'@(\w+)', '')
list_of_tagged_users = []
for x in range(len(working_df)):
    try:
        list_of_tagged_users += working_df['tagged_users'][x]
    except:
        pass



/tmp/ipykernel_786/3040538264.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  working_df['full_text'] = working_df['full_text'].str.replace(r'@(\w+)', '')


In [75]:
#Create a table for network analysis
na_df = working_df[['tweet_id','user', 'retweet', 'tagged_users']]

In [76]:
#Create a function that would create a dataframe that shows the interaction
def get_interaction(df):
    func_df = pd.DataFrame(columns = ['tweet_id', 'from_', 'to_'])
    filter_out = ['2','A','App','AppleSupport','AppleTV','AppleTVI','AppleTVPlus','AppleTVUK','BB','BBC','BBCO','BBC','BBCOn', 'BBCOne', 'BBCPlayer','BBCSounds', 'BBCR1',
                  'BBCSounds','BBCiPlayer','BBCone','D','Di','Dis','Disn', 'Disne', 'Dinsey',' DisneyChannel','DisneyHyperion','DisneyIT', 'DisneyP', 'DisneyChannel',
                  'DisneyHyperion','DisneyPlusUK', 'DisneyPlus', 'HBO', 'HBO_UK','NETFLIX', 'netflix', 'NETFLIXUK', 'Netflix', 'NetflixUK','ParamountPlus', 'ParamaountUK', 'ParamountPlusUK'
                  'Prime', 'PrimeUK', 'PrimeVideo', 'primevideouk']
    for x in range(len(df)):
        if df['retweet'][0]=='True':
            if df['tagged_users'].iloc[x][0] not in filter_out:
                new_row = {'tweet_id':df['tweet_id'].iloc[x], 'from_':df['tagged_users'].iloc[x][0], 'to_': df['user'].iloc[x]}
                func_df = func_df.append(new_row, ignore_index = True)
            else:
                print(df['tagged_users'].iloc[x][0])
        else:
            for user in df['tagged_users'].iloc[x]:
                if user not in filter_out:
                    new_row = {'tweet_id':df['tweet_id'].iloc[x], 'to_':user, 'from_': df['user'].iloc[x]}
                    func_df = func_df.append(new_row, ignore_index = True)
                else:
                    print(user)
    return func_df.drop_duplicates()


In [77]:
na_interac_df =get_interaction(na_df)
filter_out = ['2','A','App','AppleSupport','AppleTV','AppleTVI','AppleTVPlus','AppleTVUK','BB','BBC','BBCO','BBC','BBCOn', 'BBCOne', 'BBCPlayer','BBCSounds', 'BBCR1',
                  'BBCSounds','BBCiPlayer','BBCone','D','Di','Dis','Disn', 'Disne', 'Dinsey',' DisneyChannel','DisneyHyperion','DisneyIT', 'DisneyP', 'DisneyChannel',
                  'DisneyHyperion','DisneyPlusUK', 'DisneyPlus', 'HBO', 'HBO_UK','NETFLIX', 'netflix', 'NETFLIXUK', 'Netflix', 'NetflixUK','ParamountPlus', 'ParamaountUK', 'ParamountPlusUK'
                  'Prime', 'PrimeUK', 'PrimeVideo', 'primevideouk']
na_interac_df = na_interac_df[~na_interac_df['from_'].isin(filter_out)]
na_interac_df = na_interac_df[~na_interac_df['to_'].isin(filter_out)]                            
print(na_interac_df[na_interac_df['from_'].isin(filter_out)])
print(na_interac_df[na_interac_df['to_'].isin(filter_out)])

AppleTV
netflix
App
Disne
BBCOn
netflix
DisneyP
DisneyP
DisneyP
netflix
App
netflix
App
netflix
App
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NETFLIXUK
netflix
NetflixUK
NetflixUK
NetflixUK
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
netflix
Dis
Dis
Dis
Dis
Dis
Dis
primevideouk
netflix
NetflixUK
BBC
netflix
netflix
NetflixUK
netflix
netflix
netflix
netflix
netflix
netflix
NetflixUK
netflix
netflix
netflix
netflix
primevideouk
netflix
netflix
netflix
netflix
netflix
NetflixUK
NetflixUK
Netflix
NetflixUK
netflix
NetflixUK
NetflixUK
primevideouk
NetflixUK
BBC
NetflixUK
netflix
NetflixUK
Netflix
NetflixUK
NetflixUK
NetflixUK
NetflixUK
Disn
NetflixUK
NetflixUK
Disn
netflix
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK
NetflixUK

In [78]:
#get a unique user dataframe
list_of_user = na_interac_df['from_'].append(na_interac_df['to_'], ignore_index=True)
list_of_user = list(set(list_of_user))
user_df = pd.DataFrame(list_of_user, columns=['username'])
user_df['user'] = user_df['username'].apply(anonymize)
user_df=user_df.sort_values('username').reset_index()
user_df.to_csv('output/username.csv', index=False)

In [79]:
#anonymise the users from the different dataframes
na_interac_df['from_'] = na_interac_df['from_'].map(user_df.set_index('username')['user'])
na_interac_df['to_'] = na_interac_df['to_'].map(user_df.set_index('username')['user'])
working_df['user'] = working_df['user'].map(user_df.set_index('username')['user'])
na_interac_df.to_csv('output/userinteraction.csv', index=False)

In [80]:
working_df = working_df[['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'retweet', 'week']]
working_df.to_csv('output/tweets.csv', index=False)

In [81]:
set(hashtags_df['tweet_id'])==set(working_df['tweet_id'])

True

In [82]:
# remove existing database if available
if os.path.exists('output/lnctweets.db'):
    os.remove('output/lnctweets.db')
# Creating a database file for the all the output csv file
# Establish a connection to the SQLite Database
conn = sqlite3.connect('output/lnctweets.db')
# Create a cursor object to execute SQL statements:
cursor = conn.cursor()

In [83]:
def create_table(table_name, columns):
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({', '.join(columns)})"
    cursor.execute(create_table_query)

In [84]:
# Define a function to insert data into the table from a CSV file:
def insert_data(table_name, csv_file):
    with open(csv_file, 'r') as file:
        csv_data = csv.reader(file)
        next(csv_data)  # Skip the header row if necessary
        num_columns = len(next(csv_data))
        insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * num_columns)})"
        cursor.executemany(insert_query, csv_data)

In [85]:
# Specify the CSV file paths and table names for each file:
csv_files = ['output/hashtags.csv',  'output/tweets.csv', 'output/userinteraction.csv']
table_names = ['hashtags', 'tweets', 'userinteraction']

In [86]:
# Create tables and insert data for each CSV file:
for i, csv_file in enumerate(csv_files):
    table_name = table_names[i]
    with open(csv_file, 'r') as file:
        csv_data = csv.reader(file)
        columns = next(csv_data)
        data_type = ' TEXT'
        columns = [x + data_type for x in columns]
        #print(columns)
        create_table(table_name, columns)
        insert_data(table_name, csv_file)

In [87]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [88]:
# open the tweets table in the .db file
query = """
    SELECT *
    FROM userinteraction
    """
conn = sqlite3.connect('output/lnctweets.db')
print(pd.read_sql_query(query,conn))
conn.close()

        tweet_id     from_       to_
0       75c047d4  c25b9a3d  5cb9cb18
1       aab6f161  385e7052  7af41a48
2       2e91d9dc  cb33bb2d  f4b0e500
3       c8f59c0c  3cfa784a  429542c4
4       f7e7b3d6  e9a05361  e1362151
...          ...       ...       ...
157156  a6376cfa  8caa00a5  ba4f5917
157157  a6376cfa  8caa00a5  73cbea66
157158  8a640409  dc284ebb  01dac1ce
157159  1d6acb9b  62f322a1  01dac1ce
157160  2a0684c6  06b030f5  3b25e8a3

[157161 rows x 3 columns]
